In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('/kaggle/input/insurance-prediction/insurance.csv')

In [ ]:
data

In [ ]:
data.info

In [ ]:
data.describe(include='all')

In [ ]:
data['sex'].replace(to_replace=['male','female'],value=[0,1],inplace=True)

In [ ]:
data

In [ ]:
data['smoker'].replace(to_replace=['no','yes'],value=[0,1],inplace=True)

In [ ]:
data

In [ ]:
data['region'].replace(to_replace=data['region'].unique(),value=list(range(len(data['region'].unique()))),
                      inplace=True)

In [ ]:
data

In [ ]:
region_variable_ohe = np.eye(len(data['region'].unique()),len(data['region'].unique()))[data['region']]

In [ ]:
region_variable_ohe

In [ ]:
processed_data = data.drop(['region'],axis=1)

In [ ]:
processed_data

In [ ]:
region_variable_df = pd.DataFrame(data=region_variable_ohe,columns=['Region 0', 'Region 1','Region 2','Region 3'])


In [ ]:
region_variable_df

In [ ]:
processed_data = pd.concat([processed_data,region_variable_df],axis=1)

In [ ]:
processed_data

In [ ]:
gt_labels = processed_data['charges']

processed_data.drop(['charges'],axis=1,inplace=True)

In [ ]:
processed_data['charges'] = gt_labels

In [ ]:
processed_data

In [ ]:
Y = np.array(processed_data['charges']).reshape(processed_data.shape[0],1)
X_transpose = np.array(processed_data.drop(['charges'],axis=1))

In [ ]:
Y.shape

In [ ]:
X_transpose.shape

In [ ]:
X_transpose = (X_transpose - np.mean(X_transpose,axis=0))/np.std(X_transpose,axis=0)

In [ ]:
X_transpose_df = pd.DataFrame(data=X_transpose)

In [ ]:
X_transpose_df

In [ ]:
X_transpose_train = X_transpose[0:int(0.7*X_transpose.shape[0])]
X_transpose_cv = X_transpose[int(0.7*X_transpose.shape[0]):int(0.9*X_transpose.shape[0])]
X_transpose_test = X_transpose[int(0.9*X_transpose.shape[0]):]

In [ ]:
Y_train = (Y[0:int(0.7*X_transpose.shape[0])] - np.mean(Y[0:int(0.7*X_transpose.shape[0])]))/np.std(Y[0:int(0.7*X_transpose.shape[0])])
Y_cv = Y[int(0.7*X_transpose.shape[0]):int(0.9*X_transpose.shape[0])]
Y_test = Y[int(0.9*X_transpose.shape[0]):]

In [ ]:
N_train = X_transpose_train.shape[0]
m = X_transpose_train.shape[1]

# Train Linear Regression in Mini Batch 

In [ ]:
def predicted_answers(theta0,theta,X_transpose):
    return theta0 + np.matmul(X_transpose,theta)

In [ ]:
def neg_log_loss(labels,predicted_labels,theta,reg_strength):
    E = labels - predicted_labels
    return (1/labels.shape[0])*np.matmul(E.T,E) + (reg_strength*(np.linalg.norm(theta))**2/2)

In [ ]:
def fit(step_size,reg_strength):
    theta0_initial = 0
    theta_initial = np.zeros((m,1))
    epsilon = step_size
    tol = 10**(-6)
    iterations = list()
    neg_log_loss_history = list()
    iteration_number = 0
    mini_batch_size = 72
    time_steps = N_train//mini_batch_size
    epoch_counter = 0 
    invalid_value_flag = 0 
    
    while(True):
        
        for i in range(0,time_steps):
            rand_indices = np.random.choice(a=np.arange(0,N_train),size=mini_batch_size,replace=False)
            X_transpose_mini_batch = X_transpose_train[rand_indices]
            train_labels_mini_batch = Y_train[rand_indices]
            
            Predicted_answers_initial = predicted_answers(theta0_initial,theta_initial,
                                                          X_transpose_mini_batch)
            theta0_final = theta0_initial - epsilon*np.mean(Predicted_answers_initial-train_labels_mini_batch)
            theta_final = theta_initial - epsilon*((1/mini_batch_size)*np.matmul(X_transpose_mini_batch.T,
                                                                                (Predicted_answers_initial-train_labels_mini_batch))+(reg_strength*theta_initial))
            
            neg_log_loss_initial = neg_log_loss(train_labels_mini_batch,Predicted_answers_initial,
                                                theta_initial,reg_strength)
            
            Predicted_answers_final = predicted_answers(theta0_final,theta_final,X_transpose_mini_batch)
            neg_log_loss_final = neg_log_loss(train_labels_mini_batch,Predicted_answers_final,
                                             theta_final,reg_strength)
            
            if np.isnan(neg_log_loss_final[0][0]) == True:
                invalid_value_flag = 1
                break
                
            theta0_initial = theta0_final
            theta_initial = theta_final
            
            iterations.append(iteration_number)
            neg_log_loss_history.append(neg_log_loss_initial[0][0])
            
            iteration_number = iteration_number + 1
            
            #print("Iterations =",iteration_number,"Mean Squared Error Loss =",neg_log_loss_initial)
            
        epoch_counter = epoch_counter + 1
        
        #print("\n\nEpochs =",epoch_counter,"Mean Squared Error Loss =",neg_log_loss_initial,"\n\n")

        
        if abs(neg_log_loss_initial - neg_log_loss_final) < tol or invalid_value_flag == 1:
            print("\n\nEpochs =",epoch_counter,"Mean Squared Error Loss =",neg_log_loss_initial,"\n\n")
            break
            
    return {(step_size,reg_strength):[theta0_final,theta_final]}

In [ ]:
def compute_mse(labels,predicted_labels):
    E = (labels - predicted_labels)
    return (1/labels.shape[0])*np.matmul(E.T,E)

In [ ]:
def rmse(labels,trained_theta0,trained_theta,X_transpose):
    predicted_labels = predicted_answers(trained_theta0,trained_theta,X_transpose)
    return np.sqrt(compute_mse(labels,predicted_labels))

In [ ]:
grid_search_results = list()
for step_size in [0.0000001,0.000001,0.00001]:
    for reg_hyp_param in [0.0001,0.001,0.01,0.1,1,10,100]:
        grid_search_results.append(fit(step_size,reg_hyp_param))

In [ ]:
Y_cv = (Y_cv - np.mean(Y_cv))/np.std(Y_cv)

In [ ]:
Y_cv

In [ ]:
grid_search_cv_results = dict()
for configs in grid_search_results:
    trained_params = list(configs.values())
    grid_search_cv_results[tuple(configs.keys())] = rmse(Y_cv,trained_params[0][0],
                                                        trained_params[0][1],X_transpose_cv)

In [ ]:
grid_search_cv_results

In [ ]:
grid_search_train_results = dict()
for configs in grid_search_results:
    trained_params = list(configs.values())
    grid_search_train_results[tuple(configs.keys())] = rmse(Y_train,trained_params[0][0],
                                                        trained_params[0][1],X_transpose_train)

In [ ]:
grid_search_train_results

In [ ]:
random_search_results = list()
for i in range(200):
    step_size = np.random.uniform(low=10**(-7),high=10**(-4))
    reg_hyp_param = np.random.uniform(low=10**(-4),high=10**(3))
    random_search_results.append(fit(step_size,reg_hyp_param))

In [ ]:
random_search_results

In [ ]:
random_search_train_results = dict()
for configs in random_search_results:
    trained_params = list(configs.values())
    random_search_train_results[tuple(configs.keys())] = rmse(Y_train,trained_params[0][0],
                                                        trained_params[0][1],X_transpose_train)

In [ ]:
random_search_train_results

In [ ]:
random_search_cv_results = dict()
for configs in random_search_results:
    trained_params = list(configs.values())
    random_search_cv_results[tuple(configs.keys())] = rmse(Y_cv,trained_params[0][0],
                                                        trained_params[0][1],X_transpose_cv)

In [ ]:
random_search_cv_results

In [ ]:
random_search_results = list()
for i in range(10):
    step_size = np.random.uniform(low=10**(-5),high=9*(10**(-5)))
    reg_hyp_param = np.random.uniform(low=10**(-2),high=9*(10**(-2)))
    random_search_results.append(fit(step_size,reg_hyp_param))

In [ ]:
random_search_cv_results = dict()
for configs in random_search_results:
    trained_params = list(configs.values())
    random_search_cv_results[tuple(configs.keys())] = rmse(Y_cv,trained_params[0][0],
                                                        trained_params[0][1],X_transpose_cv)

In [ ]:
random_search_train_results = dict()
for configs in random_search_results:
    trained_params = list(configs.values())
    random_search_train_results[tuple(configs.keys())] = rmse(Y_train,trained_params[0][0],
                                                        trained_params[0][1],X_transpose_train)

In [ ]:
random_search_train_results

In [ ]:
random_search_cv_results